### Reading this article for understanding

https://diegoquintanav.github.io/dq-de-mission-01-EN.html

In [1]:
!pwd

/Users/mykras/Jupyter Notebooks


In [2]:
!ls -GFlash --color /Users/mykras/Downloads/docker_psycopg2_postgres-main

total 24
0 drwxr-xr-x@  5 mykras  staff   160B Nov  2 04:04 ./
0 drwx------@ 74 mykras  staff   2.3K Nov  2 12:04 ../
8 -rw-r--r--@  1 mykras  staff   224B Nov  2 04:04 Commands.sh
8 -rw-r--r--@  1 mykras  staff    96B Nov  2 04:04 README.md
8 -rw-r--r--@  1 mykras  staff   633B Nov  2 04:04 docker-compose.yml


In [3]:
%cd /Users/mykras/Downloads/docker_psycopg2_postgres-main
!ls -GFlash 

/Users/mykras/Downloads/docker_psycopg2_postgres-main
total 24
0 drwxr-xr-x@  5 mykras  staff   160B Nov  2 04:04 ./
0 drwx------@ 74 mykras  staff   2.3K Nov  2 12:04 ../
8 -rw-r--r--@  1 mykras  staff   224B Nov  2 04:04 Commands.sh
8 -rw-r--r--@  1 mykras  staff    96B Nov  2 04:04 README.md
8 -rw-r--r--@  1 mykras  staff   633B Nov  2 04:04 docker-compose.yml


In [4]:
!cat Commands.sh

!cat docker-compose.yml

!docker-compose -p dq_de up -d

# check if postgres docker instance is alive
!docker ps

# retrieve the ip address of the container, seen from the host
#!docker inspect dq_de_db_1 | grep "IPAddress"


In [5]:
!cat docker-compose.yml

# Use postgres/example user/password credentials
version: '3.1'

services:
  db:
    image: postgres:14
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: postgres 
    volumes:
      - pgdata:/var/lib/postgresql/data
    ports:
      - 54320:5432
    networks:
      db_nw:
        aliases:
          - postgres
  pgadmin:
    image: dpage/pgadmin4:latest
    environment:
      POSTGRES_PASSWORD: postgres 
      PGADMIN_DEFAULT_EMAIL: user@domain.com 
      PGADMIN_DEFAULT_PASSWORD: admin
    ports:
      - 8080:80
    networks:
      - db_nw

volumes:
    pgdata: {}

networks:
  db_nw:
    driver: bridge


In [6]:
#
!docker-compose -p dq_de up -d

[+] Running 0/0
 ⠋ Network dq_de_db_nw  Creating                                           0.0s
[+] Running 2/1
 ⠿ Network dq_de_db_nw        Created                                      0.0s
 ⠿ Volume "dq_de_pgdata"      Created                                      0.0s
 ⠿ Container dq_de-db-1       Starting                                     0.1s
 ⠿ Container dq_de-pgadmin-1  Starting                                     0.1s
[+] Running 2/4
 ⠿ Network dq_de_db_nw        Created                                      0.0s
 ⠿ Volume "dq_de_pgdata"      Created                                      0.0s
 ⠿ Container dq_de-db-1       Starting                                     0.2s
 ⠿ Container dq_de-pgadmin-1  Starting                                     0.2s
[+] Running 2/4
 ⠿ Network dq_de_db_nw        Created                                      0.0s
 ⠿ Volume "dq_de_pgdata"      Created                                      0.0s
 ⠿ Container dq_de-db-1       Starting                  

In [7]:
!docker ps

CONTAINER ID   IMAGE                   COMMAND                  CREATED         STATUS         PORTS                           NAMES
7be0e20c63bb   postgres:14             "docker-entrypoint.s…"   4 seconds ago   Up 3 seconds   0.0.0.0:54320->5432/tcp         dq_de-db-1
d79ac76b29a5   dpage/pgadmin4:latest   "/entrypoint.sh"         4 seconds ago   Up 3 seconds   443/tcp, 0.0.0.0:8080->80/tcp   dq_de-pgadmin-1


In [8]:
# CONTAINER ID NAMES for dq_de-db-1
!docker inspect 7be0e20c63bb | grep "IPAddress"

            "SecondaryIPAddresses": null,
            "IPAddress": "",
                    "IPAddress": "172.19.0.2",


In [14]:
#pip install --upgrade pip
#!pip3 install ddlgenerator
import pandas as pd
import glob, os
import psycopg2
from ddlgenerator.ddlgenerator import Table

In [12]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres", user="postgres", password="postgres", host="172.19.0.2"
)

conn.autocommit = True
cur = conn.cursor()

DATA_PROD_USER_PWD = "changeme"
DATA_VIEW_USER_PWD = "changeme"

# create user and detach password from command
# also add an IF NOT EXIST clause as a hack from
# https://stackoverflow.com/questions/8092086/create-postgresql-role-user-if-it-doesnt-exist
cur.execute(f"""
CREATE USER data_prod_user WITH PASSWORD '{DATA_PROD_USER_PWD}';
CREATE GROUP readonly WITH NOLOGIN;
CREATE USER analyst1 WITH PASSWORD '{DATA_VIEW_USER_PWD}' IN GROUP readonly;
""")

# create db
print("CREATE DATABASE ihw OWNER postgres")
cur.execute("CREATE DATABASE ihw OWNER postgres")

conn.commit()
conn.close()

OperationalError: connection to server at "172.19.0.2", port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?
